In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
rent_df = pd.read_excel('../../data/sale/sale.xlsx')
index_df = pd.read_excel('../../data/price_index.xlsx')

In [3]:
index_df = index_df.replace({'\xa0': ' '}, regex=True)
index_df = index_df[['районы', 'метро']]

In [4]:
index_df = index_df.dropna()
reverse_index = dict()

for idx, series in index_df.iterrows():
    metros = series['метро'].split(', ')
    for metro in metros:
        reverse_index[metro] = series['районы']

reverse_index['Красный Балтиец'] = reverse_index['Сокол']
reverse_index['Бескудниково'] = reverse_index['Савеловская']
reverse_index['Петровский Парк'] = reverse_index['Динамо']
reverse_index['Гражданская'] = reverse_index['Динамо']
reverse_index['Дегунино'] = reverse_index['Ховрино']
reverse_index['Марк'] = reverse_index['Алтуфьево']

In [5]:
def extract_district(m: str) -> str:
    return reverse_index[m]

rent_df['district'] = rent_df['metro'].apply(extract_district)

In [6]:
mean_rent = rent_df.groupby(by=['district'])['price'].mean()

In [7]:
def assign_mean(m: str):
    return mean_rent[m]

rent_df['mean_district'] = rent_df['district'].apply(assign_mean)

In [8]:
rent_df['district_adjustment'] = mean_rent['Левобережный, Ховрино'] / rent_df['mean_district']

In [9]:
mean_ratio = mean_rent['Левобережный, Ховрино'] / mean_rent

mean_ratio.to_excel('../../data/sale/mean_ratio.xlsx')

rent_df.to_excel('../../data/sale/distr_adjustment.xlsx')